In [34]:
from sort import Sort
import os
import numpy as np

def load_detections(detection_file):
    detections = []
    with open(detection_file, "r") as f:
        for line in f:
            parts = line.strip().split(',')
            frame_number = int(parts[0])
            bbox = [float(part) for part in parts[2:6]]  # x, y, width, height
            confidence = float(parts[6])
            detections.append((frame_number, bbox, confidence))
    return detections

def detect_and_track(detections_path, output_path):
    mot_tracker = Sort()
    detection_file = os.path.join(detections_path, "det", "det.txt")  # Adjust based on your file structure
    
    detections = load_detections(detection_file)
    current_frame = 1
    output_file_path = os.path.join(output_path, "tracking_output.txt")
    
    with open(output_file_path, "w") as output_file:
        for frame_number, bbox, confidence in detections:
            while current_frame < frame_number:
                mot_tracker.update([])  # Update tracker with no detections for skipped frames
                current_frame += 1

    
            
            # Convert bbox from [x,y,w,h] to [x1,y1,x2,y2] for SORT
            x1, y1, w, h = bbox
            x2, y2 = x1 + w, y1 + h
            sort_detection = np.array([[x1, y1, x2, y2, confidence]])

            # print(sort_detection)
            
            track_bbs_ids = mot_tracker.update(sort_detection)
            
            for track in track_bbs_ids:
                bbox, track_id = track[:4], int(track[4])
                output_file.write(f"{frame_number},{track_id},{bbox[0]},{bbox[1]},{bbox[2]-bbox[0]},{bbox[3]-bbox[1]},-1,-1,-1,-1\n")
                
            current_frame += 1

# Example usage
detections_path = r'C:\Users\p_ayu\OneDrive\Desktop\CV\MOT17\MOT17\train\MOT17-05-DPM'  # Adjust to your path
output_path = r'C:\Users\p_ayu\OneDrive\Desktop\CV'
# C:\Users\p_ayu\OneDrive\Desktop\CV

detect_and_track(detections_path,output_path)

In [35]:
def load_detections(detection_file):
    detections = {}
    with open(detection_file, "r") as f:
        for line in f:
            parts = line.strip().split(',')
            frame_number = int(parts[0])
            bbox = [float(part) for part in parts[2:6]]  # x, y, width, height
            confidence = float(parts[6])
            if frame_number not in detections:
                detections[frame_number] = []
            detections[frame_number].append((bbox, confidence))
    return detections

def load_ground_truth(ground_truth_file):
    ground_truth = {}
    with open(ground_truth_file, "r") as f:
        for line in f:
            parts = line.strip().split(',')
            frame_number = int(parts[0])
            track_id = int(parts[1])
            bbox = [float(part) for part in parts[2:6]]  # x, y, width, height
            if frame_number not in ground_truth:
                ground_truth[frame_number] = {}
            ground_truth[frame_number][track_id] = bbox
    return ground_truth

def calculate_mota_motp(tracking_results, ground_truth):
    gt_tracks = {}
    tp = fp = fn = mt = mme = 0

    for frame_number, tr_frame in tracking_results.items():
        if frame_number in ground_truth:
            gt_frame = ground_truth[frame_number]

            for tr_bbox, tr_confidence in tr_frame:
                best_iou = 0
                best_gt_id = None

                for gt_track_id, gt_bbox in gt_frame.items():
                    iou = calculate_iou(tr_bbox, gt_bbox)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_id = gt_track_id

                if best_iou >= 0.5:
                    if best_gt_id in gt_tracks:
                        if gt_tracks[best_gt_id] == tr_bbox:
                            continue
                        else:
                            mme += 1
                            gt_tracks[best_gt_id] = tr_bbox
                            fp -= 1
                    else:
                        gt_tracks[best_gt_id] = tr_bbox
                    tp += 1
                else:
                    fp += 1

            fn += len(gt_frame) - len(gt_tracks)
            mt += len(gt_tracks)

    if tp + fp + fn == 0:
        mota = 1
    else:
        mota = 1 - (fn + fp + mme) / (tp + fp + fn)
    
    if tp == 0:
        motp = 0
    else:
        motp = mt / tp
    
    return mota, motp

# Example usage
tracking_results_path = r'C:\Users\p_ayu\OneDrive\Desktop\CV\tracking_output.txt'  # Path to your tracking output file
ground_truth_path = r'C:\Users\p_ayu\OneDrive\Desktop\CV\MOT17\MOT17\train\MOT17-05-DPM\gt\gt.txt'  # Path to your ground truth file

tracking_results = load_detections(tracking_results_path)
ground_truth = load_ground_truth(ground_truth_path)

mota, motp = calculate_mota_motp(tracking_results, ground_truth)
print("MOTA:", mota)
print("MOTP:", motp)


MOTA: 0.02631578947368418
MOTP: 4.206896551724138
